In [2]:
from dataclasses import dataclass
from pathlib import Path

import pandas as pd

from mmvae_hub.base.experiment_vis.utils import *
from mmvae_hub.base.utils.MongoDB import MongoDatabase
from mmvae_hub.base.utils.flags_utils import get_config_path
from mmvae_hub.base.utils.utils import dict2pyobject, json2dict

[19:29:59 CEST] Log file: /usr/local/var/logs/mmvae_hub/UTC-20210516-172959.log


In [3]:
exp_db = MongoDatabase(training=False)
experiments = exp_db.connect()

In [4]:
print('There are currently ', experiments.count_documents({}), 'experiments in the database.')

There are currently  24 experiments in the database.


In [5]:
df = pd.DataFrame()
# Find all experiments with method joint_elbo
# for exp in experiments.find({'flags.method':'joint_elbo'}):
for exp in experiments.find({}):
    if exp['epoch_results'] is not None and exp['epoch_results']:
        last_epoch = str(max([int(epoch) for epoch in exp['epoch_results']]))
        last_epoch_results = exp['epoch_results'][last_epoch]['test_results']

        if exp['flags']['method'] == 'planar_mixture':
            exp['flags']['method'] = f"{exp['flags']['method']}_{exp['flags']['num_flows']}"

        results_dict = {**exp['flags'], 'end_epoch': last_epoch, '_id': exp['_id']}

        scores = []
        scores_lr = []
        scores_gen = []
        scores_prd = []

        if last_epoch_results['lr_eval'] and last_epoch_results['gen_eval']:

            # get lr_eval results
            for key, val in last_epoch_results['lr_eval'].items():
                results_dict[f'lr_eval_{key}'] = val['accuracy']
                scores.append(val['accuracy'])
                scores_lr.append(val['accuracy'])

            # get gen_eval results
            for key, val in last_epoch_results['gen_eval'].items():
                key = key.replace('digit_', '')
                results_dict[f'gen_eval_{key}'] = val
                scores.append(val)
                scores_gen.append(val)

            if 'prd_scores' in last_epoch_results and last_epoch_results['prd_scores']:
                for key, val in last_epoch_results['prd_scores'].items():
                    results_dict[f'prd_score_{key}'] = val
                    scores_prd.append(val)

            results_dict['score'] = sum(scores)
            results_dict['score_lr'] = sum(scores_lr)
            results_dict['score_gen'] = sum(scores_gen)
            results_dict['score_prd'] = sum(scores_prd)
            df = df.append(results_dict, ignore_index=True)

In [6]:
show_cols = ['score', 'method', 'end_epoch', 'class_dim', 'beta', '_id', 'score_lr', 'score_gen','score_prd', 'num_mods']
df[show_cols].sort_values(by=['score'], ascending=False)

,score,method,end_epoch,class_dim,beta,_id,score_lr,score_gen,score_prd,num_mods
0,15.368883,joint_elbo,299,64.0,2.5,polymnist_joint_elbo_2021_05_04_09_21_34_487039,5.252283,10.116600,0.000000,3.0
18,13.342450,moe,299,512.0,2.5,polymnist_moe_2021_05_16_08_11_11_173474,4.227050,9.115400,0.433640,3.0
4,11.739267,joint_elbo,299,64.0,0.5,polymnist_joint_elbo_2021_05_08_05_21_05_683057,3.932350,7.806917,0.611546,3.0
15,11.271117,planar_mixture_5,299,512.0,2.5,polymnist_planar_mixture_2021_05_13_15_20_43_7...,1.477533,9.793583,2.595059,3.0
13,11.132117,planar_mixture_5,299,512.0,2.5,polymnist_planar_mixture_2021_05_12_11_38_58_1...,1.349800,9.782317,1.495992,3.0
5,11.092400,moe,299,128.0,0.5,polymnist_moe_2021_05_09_01_46_10_858142,3.572933,7.519467,0.332367,3.0
2,9.968633,joint_elbo,299,32.0,0.5,polymnist_joint_elbo_2021_05_06_12_59_58_785343,3.239317,6.729317,0.312330,3.0
3,7.705983,moe,299,64.0,0.5,polymnist_moe_2021_05_07_09_09_37_528857,2.489417,5.216567,0.192170,3.0
1,6.143800,moe,299,32.0,0.5,polymnist_moe_2021_05_05_17_04_47_279351,1.717367,4.426433,0.180041,3.0
8,0.771817,moe,299,512.0,2.5,polymnist_moe_2021_05_10_21_53_22_903726,0.185717,0.586100,0.229577,1.0


Generate samples with the model that has the highest score.

In [7]:
best_score_id = df[df.score == df.score.max()]._id.item()
print(best_score_id)
experiments_database = MongoDatabase(training=False, _id=best_score_id)
experiment_dict = experiments_database.get_experiment_dict()

config = json2dict(get_config_path())
experiment_flags = experiment_dict['flags']

for key in config:
    experiment_flags[key] = config[key]

experiment_flags['use_clf'] = False
experiment_flags['device'] = 'cpu'



polymnist_joint_elbo_2021_05_04_09_21_34_487039


In [8]:
show_generated_figs(flags=dict2pyobject(experiment_flags))

JointElboMMVae()


RuntimeError: Error(s) in loading state_dict for EncoderImg:
	Missing key(s) in state_dict: "class_logvar.0.weight", "class_logvar.0.bias". 
	Unexpected key(s) in state_dict: "class_logvar.weight", "class_logvar.bias". 